In [176]:
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import cufflinks as cf

In [177]:
nl_fac = pd.read_csv(r'C:\Users\David\Python\scrapy\facup\02_merge\nleague_teams_v1.csv')
nl_tab = pd.read_csv(r'C:\Users\David\Python\scrapy\facup\01_league_tables\nonleague_1952-89_v3.csv')

In [178]:
# restricting missing teams to range of scraped tables

nl_fac = nl_fac[nl_fac.year < 1990]

In [179]:
nl_fac.head()

,team,year,round,stage
0,AYLESBURY UNITED,1952,FIRST ROUND PROPER,8
1,BANGOR CITY,1952,FIRST ROUND PROPER,8
2,BARNSTAPLE TOWN,1952,FIRST ROUND PROPER,8
3,BARRY TOWN,1952,FIRST ROUND PROPER,8
4,BEDFORD TOWN,1952,FIRST ROUND PROPER,8


In [180]:
nl_tab.head()

,year,pos,league,team
0,1952,1,ATHENIAN LEAGUE,WEALDSTONE
1,1952,2,ATHENIAN LEAGUE,HENDON
2,1952,3,ATHENIAN LEAGUE,BROMLEY
3,1952,4,ATHENIAN LEAGUE,SUTTON U.
4,1952,5,ATHENIAN LEAGUE,HAYES


In [181]:
nl_fac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1216 entries, 0 to 1215
Data columns (total 4 columns):
team     1216 non-null object
year     1216 non-null int64
round    1216 non-null object
stage    1216 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 47.5+ KB


In [182]:
nl_tab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32533 entries, 0 to 32532
Data columns (total 4 columns):
year      32533 non-null int64
pos       32533 non-null int64
league    32533 non-null object
team      32533 non-null object
dtypes: int64(2), object(2)
memory usage: 1.2+ MB


####  Merge test 1

In [183]:
nl_merge = pd.merge(nl_tab, nl_fac, how='right', on=['team', 'year'])

#### Plotting missing matches, should be 32 matches per year

In [184]:
missing_teams = nl_merge[nl_merge.pos.isnull()].copy()

In [185]:
mt_plot = missing_teams.year.value_counts().sort_index().reindex()

In [186]:
mt_plot.iplot(kind='bar', dimensions = (700,400))

In [187]:
missing_teams = missing_teams[missing_teams.year < 1990]

#### Only 1962 has 32 matches, apply fuzzy string matching to try and find near matches

In [188]:
from fuzzywuzzy import process
import fuzzywuzzy
import Levenshtein

In [189]:
def near_match(row):
    return process.extractOne(row['team'], nl_tab[nl_tab.year == row['year']]['team'].values) # only search specific year
     #return tuple containing (nearest match, score)
    
%time missing_teams['match'], missing_teams['score'] = zip(*missing_teams.apply(near_match, axis = 1))

Wall time: 22 s


In [190]:
missing_teams[missing_teams.score > 90].sort_values('score')

,year,pos,league,team,round,stage,match,score
1260,1953,NaN,NaN,PETERBOROUGH UNITED,FIRST ROUND PROPER,8,PETERBOROUGH UTD.,91
1262,1953,NaN,NaN,WALTHAMSTOW AVENUE,FIRST ROUND PROPER,8,WALTHAMSTOW AVE.,91
1279,1954,NaN,NaN,PETERBOROUGH UNITED,FIRST ROUND PROPER,8,PETERBOROUGH UTD.,91
1281,1954,NaN,NaN,WALTHAMSTOW AVENUE,FIRST ROUND PROPER,8,WALTHAMSTOW AVE.,91
1298,1955,NaN,NaN,WALTHAMSTOW AVENUE,FIRST ROUND PROPER,8,WALTHAMSTOW AVE.,91
1377,1976,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1373,1975,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1383,1978,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1387,1979,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1328,1964,NaN,NaN,TOOTING & MITCHAM UNITED,FIRST ROUND PROPER,8,TOOTING & MITCHAM UTD.,93


Checking that no missing FA cup teams end with UTD. then then replacing UTD. with UNITED for non-league teams

In [191]:
nl_fac[nl_fac.team.str.contains('UTD\.$', regex=True)].team.unique()

array([], dtype=object)

In [192]:
nl_tab.team = nl_tab.team.str.replace('UTD\.$', 'UNITED')

Checking that no missing FA cup teams end with AVE. then then replacing AVE. with AVENUE for non-league teams

In [193]:
nl_fac[nl_fac.team.str.contains('AVE\.$', regex=True)].team.unique()

array([], dtype=object)

In [194]:
nl_tab.team = nl_tab.team.str.replace('AVE\.$', 'AVENUE')

Checking that no missing FA cup teams end . then then replacing . with '' for non-league teams

In [195]:
nl_fac[nl_fac.team.str.contains('\.$', regex=True)].team.unique()

array([], dtype=object)

In [196]:
nl_tab.team = nl_tab.team.str.replace('\.$', '')

#### Merge test 2

In [197]:
nl_merge2 = pd.merge(nl_tab, nl_fac, how='right', on=['team', 'year'])

In [198]:
missing_teams2 = nl_merge2[nl_merge2.pos.isnull()].copy()

In [199]:
mt_plot2 = missing_teams2.year.value_counts().sort_index().reindex()

In [200]:
#making dataframe which join missing teams from 1st and 2nd merge

mt_plot_join = pd.DataFrame({'mt_plot':mt_plot, 'mt_plot2':mt_plot2})

mt_plot_join.iplot(kind='bar', barmode='overlay', dimensions = (700,400))

In [201]:
%time missing_teams2['match'], missing_teams2['score'] = zip(*missing_teams2.apply(near_match, axis = 1))

missing_teams2[missing_teams2.score > 90].sort_values('score')

Wall time: 17.4 s


,year,pos,league,team,round,stage,match,score
1391,1978,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1384,1975,NaN,NaN,TOOTING & MITCHAM UNITED,FIRST ROUND PROPER,8,TOOTING & MITCHAM UTD,93
1381,1975,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1395,1979,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1399,1980,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1385,1976,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1394,1978,NaN,NaN,TOOTING & MITCHAM UNITED,FIRST ROUND PROPER,8,TOOTING AND MITCHAM UNITED,95
1401,1980,NaN,NaN,LEYTONSTONE/ILFORD,FIRST ROUND PROPER,8,LEYTONSTONE & ILFORD,95
1270,1953,NaN,NaN,HEREFORD UNITED,FIRST ROUND PROPER,8,HEREFORD UNITED RES,95
1412,1984,NaN,NaN,FARNBOROUGH,FIRST ROUND PROPER,8,FARNBOROUGH TOWN,95


#### Checking for reserve leagues

Want to drop any leagues with RES in the title and reserve teams cannot qualify for FA Cup

First finding any current matches that are matched to a team from a reserve league and don't have multiple matches

i.e. any team where there is a reserve team matched but we have no match for the first team

In [202]:
nl_merge2.groupby(['year', 'team']).filter(lambda x: (len(x) < 2) & (x['league'].str.contains('RES', regex=True)).any())

,year,pos,league,team,round,stage
1151,1986,2,RESERVE TEAM FOOTBALL - ESSEX & HERTS BORDER C...,LEYTON-WINGATE,FIRST ROUND PROPER,8


In [203]:
nl_tab[(nl_tab.team.str.contains('LEYTON')) & (nl_tab.year == 1986)]

,year,pos,league,team
23516,1986,6,ISTHMIAN LEAGUE DIVISION 1,LEYTON WINGATE
23522,1986,12,ISTHMIAN LEAGUE DIVISION 1,LEYTONSTONE & ILFORD
24251,1986,11,THE OLD BOYS AMATEUR FOOTBALL LEAGUE SENIOR DI...,LEYTON COUNTY O.B
24292,1986,2,RESERVE TEAM FOOTBALL - ESSEX & HERTS BORDER C...,LEYTON-WINGATE


It should be safe to drop ALL reserve leagues as real LEYTON-WINGATE team exists in ISTHMIAN league and match will be made later

First extracting the strings containing RES in the league names to check they all refer to reserve leagues

In [204]:
nl_tab.league.str.extract('(RES.*)').unique()

array([nan, 'RESERVE TEAM COMPETITION MIDDLESEX BORDER LEAGUE',
       'RESERVE TEAM COMPETITIONS SUBURBAN LEAGUE NORTH',
       'RESERVE TEAM COMPETITION SUBURBAN LEAGUE SOUTH',
       'REST COMBINATION LEAGUE', 'RESERVES DIVISION 1',
       'RESERVE TEAM FOOTBALL - ESSEX & HERTS BORDER COMBINATION EASTERN DIVISION',
       'RESERVE TEAM FOOTBALL - ESSEX & HERTS BORDER COMBINATION WESTERN DIVISION',
       'RESERVE TEAM FOOTBALL - MIDDLESEX BORDER COMBINATION',
       'RESERVE TEAM FOOTBALL - SURBURBAN LEAGUE NORTH',
       'RESERVE TEAM FOOTBALL - SURBURBAN LEAGUE SOUTH',
       'RESERVE DIVISION', 'RESERVE TEAM FOOTBALL - SURBURBAN LEAGUE WEST',
       'RESERVE DIVISION ONE', 'RESERVE DIVISION TWO'], dtype=object)

Need to preserve 'REST COMBINATION LEAGUE' by looking for 'RESERVE' rather than 'RES'

#### Dropping reserve leagues

In [205]:
nl_tab = nl_tab[~nl_tab.league.str.contains('RESERVE', regex=True)]

#### Checking for remaining 'reserve' teams

Now checking for teams containing RES in title in FA cup teams

In [206]:
nl_fac[nl_fac.team.str.contains(' RES', regex=True)]

,team,year,round,stage


Now checking for teams containing RES in title in Non league tables

In [207]:
nl_tab.team.str.extract('( RES.*)').unique()

array([nan, ' RES', ' RESERVES', ' RESERVE', ' RESERBES', ' RESEARCH',
       ' RESERVES OB'], dtype=object)

No FA cup teams contain RES (see above) so happy to drop RESEARCH also

#### Dropping reserve teams

In [208]:
nl_tab = nl_tab[~nl_tab.team.str.contains(' RES', regex=True)]

In [209]:
%time missing_teams2['match'], missing_teams2['score'] = zip(*missing_teams2.apply(near_match, axis = 1))

missing_teams2[missing_teams2.score > 90].sort_values('score')

Wall time: 15.7 s


,year,pos,league,team,round,stage,match,score
1381,1975,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1384,1975,NaN,NaN,TOOTING & MITCHAM UNITED,FIRST ROUND PROPER,8,TOOTING & MITCHAM UTD,93
1385,1976,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1391,1978,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1399,1980,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1395,1979,NaN,NaN,AP LEAMINGTON,FIRST ROUND PROPER,8,A.P. LEAMINGTON,93
1271,1953,NaN,NaN,HORDEN CW,FIRST ROUND PROPER,8,HORDEN C.W,95
1423,1986,NaN,NaN,FARNBOROUGH,FIRST ROUND PROPER,8,FARNBOROUGH TOWN,95
1422,1985,NaN,NaN,WINDSOR & ETON,FIRST ROUND PROPER,8,WINDSOR AND ETON,95
1421,1985,NaN,NaN,V.S. RUGBY,FIRST ROUND PROPER,8,V.S.RUGBY,95


After dropping reserve teams I am happy that all the teams with near matches (>90) in this list are correct so creating a series to map the names in nl_tab to the actual names in nl_fac

In [210]:
miss_90 = missing_teams2[missing_teams2.score > 90].sort_values('score')

In [211]:
name_map1 = pd.Series(data=miss_90.team.values, index=miss_90.match.values)

name_map1

A.P. LEAMINGTON                          AP LEAMINGTON
TOOTING & MITCHAM UTD         TOOTING & MITCHAM UNITED
A.P. LEAMINGTON                          AP LEAMINGTON
A.P. LEAMINGTON                          AP LEAMINGTON
A.P. LEAMINGTON                          AP LEAMINGTON
A.P. LEAMINGTON                          AP LEAMINGTON
HORDEN C.W                                   HORDEN CW
FARNBOROUGH TOWN                           FARNBOROUGH
WINDSOR AND ETON                        WINDSOR & ETON
V.S.RUGBY                                   V.S. RUGBY
KINGS LYNN                                 KING'S LYNN
FARNBOROUGH TOWN                           FARNBOROUGH
FARNBOROUGH TOWN                           FARNBOROUGH
HORDEN C.W                                   HORDEN CW
FARNBOROUGH TOWN                           FARNBOROUGH
LEYTONSTONE & ILFORD                LEYTONSTONE/ILFORD
TOOTING AND MITCHAM UNITED    TOOTING & MITCHAM UNITED
TOOTING & MITCHAM             TOOTING & MITCHAM UNITED
TOOTING & 

#### Updating team names in nl_tab

In [212]:
nl_tab.team.replace(to_replace=name_map1.index, value=name_map1.values, inplace=True)

#### Merge test 3

In [213]:
nl_merge3 = pd.merge(nl_tab, nl_fac, how='right', on=['team', 'year'])

In [214]:
missing_teams3 = nl_merge3[nl_merge3.pos.isnull()].copy()

mt_plot3 = missing_teams3.year.value_counts().sort_index().reindex()

In [215]:
mt_plot_join['mt_plot3'] = mt_plot3

mt_plot_join.iplot(kind='bar', barmode='overlay', dimensions = (700,400))

In [216]:
%time missing_teams3['match'], missing_teams3['score'] = zip(*missing_teams3.apply(near_match, axis = 1))


# widening fuzzy string match tolerance to above 88
missing_teams3[missing_teams3.score > 88].sort_values('score')

Wall time: 13.7 s


,year,pos,league,team,round,stage,match,score
1306,1955,NaN,NaN,TUNBRIDGE WELLS UNITED,FIRST ROUND PROPER,8,TUNBRIDGE WELLS U,89
1269,1953,NaN,NaN,BEIGHTON MINERS WELFARE,FIRST ROUND PROPER,8,BEIGHTON MINERS W,89
1405,1988,NaN,NaN,V.S. RUGBY,FIRST ROUND PROPER,8,VS RUGBY,89
1296,1955,NaN,NaN,CHELMSFORD CITY,FIRST ROUND PROPER,8,CHELMSFORD C,89
1402,1987,NaN,NaN,V.S. RUGBY,FIRST ROUND PROPER,8,VS RUGBY,89
1399,1986,NaN,NaN,V.S. RUGBY,FIRST ROUND PROPER,8,VS RUGBY,89
1320,1959,NaN,NaN,TOOTING & MITCHAM UNITED,FIRST ROUND PROPER,8,TOOTING & MITCHAM U,89
1366,1973,NaN,NaN,GRANTHAM TOWN,FIRST ROUND PROPER,8,GRANTHAM,90
1365,1973,NaN,NaN,ALTON TOWN,FIRST ROUND PROPER,8,LUTON TOWN,90
1359,1971,NaN,NaN,GRANTHAM TOWN,FIRST ROUND PROPER,8,GRANTHAM,90


Have checked each of the fuzzy matches is a legitmate match (e.g. checked other sources / dates on teams of similar name existing)

The only ones in this list that are not matches properly are LUTON TOWN, FLEET, CHESTER and SANKEYS (WELLINGTON)

In [217]:
no_match = ['LUTON TOWN', 'FLEET', 'CHESTER', 'SANKEYS (WELLINGTON)']

miss_88 = missing_teams3[(missing_teams3.score > 88) & ~(missing_teams3.match.isin(no_match))]

In [218]:
name_map2 = pd.Series(data=miss_88.team.values, index=miss_88.match.values)

name_map2

BILLINGHAM                 BILLINGHAM SYNTHONIA
TONBRIDGE                      TONBRIDGE ANGELS
BEIGHTON MINERS W       BEIGHTON MINERS WELFARE
HEREFORD                        HEREFORD UNITED
KIDDERMINSTER            KIDDERMINSTER HARRIERS
NETHERFIELD                NETHERFIELD (KENDAL)
TONBRIDGE                      TONBRIDGE ANGELS
WELLINGTON TOWN                      WELLINGTON
YEOVIL                              YEOVIL TOWN
CAMBRIDGE UTD                  CAMBRIDGE UNITED
HASTINGS U                      HASTINGS UNITED
HEREFORD U                      HEREFORD UNITED
WELLINGTON TOWN                      WELLINGTON
CAMBRIDGE UTD                  CAMBRIDGE UNITED
CHELMSFORD C                    CHELMSFORD CITY
HASTINGS U                      HASTINGS UNITED
NETHERFIELD                NETHERFIELD (KENDAL)
TUNBRIDGE WELLS U        TUNBRIDGE WELLS UNITED
WELLINGTON TOWN                      WELLINGTON
NETHERFIELD                NETHERFIELD (KENDAL)
BILLINGHAM                 BILLINGHAM SY

#### Updating team names in nl_tab

In [219]:
nl_tab.team.replace(to_replace=name_map2.index, value=name_map2.values, inplace=True)

#### Merge test 4

In [220]:
nl_merge4 = pd.merge(nl_tab, nl_fac, how='right', on=['team', 'year'])

missing_teams4 = nl_merge4[nl_merge4.pos.isnull()].copy()

mt_plot4 = missing_teams4.year.value_counts().sort_index().reindex()

In [221]:
mt_plot_join['mt_plot4'] = mt_plot4

mt_plot_join.iplot(kind='bar', barmode='overlay', dimensions = (700,400))

In [222]:
mt_plot.sum()

188L

In [223]:
mt_plot4.sum()

90L

Through regulariasation and fuzzy string matching with QC have found 100 extra matches :)

Fuzzy matches below a score of 88 were quite erratic so returning multiple potential matches per team with new function

In [224]:
missing_teams4[missing_teams4.year == 1952]

,year,pos,league,team,round,stage
1331,1952,NaN,NaN,BLACKHALL COLLIERY WELFARE,FIRST ROUND PROPER,8
1332,1952,NaN,NaN,FOLKESTONE,FIRST ROUND PROPER,8
1333,1952,NaN,NaN,MERTHYR TOWN,FIRST ROUND PROPER,8
1334,1952,NaN,NaN,RAWMARSH WELFARE,FIRST ROUND PROPER,8


Creating new string matching function that returns multiple matches

In [225]:
def near_matches(row):
    matches =  process.extract(row['team'], nl_tab[nl_tab.year == row['year']]['team'].values, limit = 3) # only search specific year
    # matches is list of tuples in form [('1st nearest match, 1st nearest match score), (2nd nearest match, 2nd nearest match score)]
    return matches

%time missing_teams4['match1'], missing_teams4['match2'], missing_teams4['match3'] = zip(*missing_teams4.apply(near_matches, axis = 1))

Wall time: 7.76 s


In [226]:
missing_teams4

,year,pos,league,team,round,stage,match1,match2,match3
1331,1952,NaN,NaN,BLACKHALL COLLIERY WELFARE,FIRST ROUND PROPER,8,"(WHITWICK COLLIERY, 86)","(LINBY COLLIERY, 86)","(CRESWELL COLLIERY, 86)"
1332,1952,NaN,NaN,FOLKESTONE,FIRST ROUND PROPER,8,"(GORLESTON, 74)","(LEYTONSTONE, 67)","(LEYTON, 63)"
1333,1952,NaN,NaN,MERTHYR TOWN,FIRST ROUND PROPER,8,"(LYE TOWN, 86)","(WELLINGBOROUGH TOWN, 86)","(SWINDON TOWN COLTS, 86)"
1334,1952,NaN,NaN,RAWMARSH WELFARE,FIRST ROUND PROPER,8,"(S. NORMANTON WELFARE, 66)","(ALTOFTS WELFARE, 65)","(BENTINCK WELFARE, 58)"
1335,1953,NaN,NaN,GREAT YARMOUTH TOWN,FIRST ROUND PROPER,8,"(HITCHIN TOWN, 86)","(LYE TOWN, 86)","(RUGBY TOWN, 86)"
1336,1953,NaN,NaN,LLANELLI,FIRST ROUND PROPER,8,"(LLANELLY, 88)","(BRIERLEY HILL ALLIANCE, 68)","(CRESWELL COLLIERY, 57)"
1337,1953,NaN,NaN,NEWPORT (IOW),FIRST ROUND PROPER,8,"(LETCHWORTH TOWN, 59)","(LETCHWORTH TOWN, 59)","(NEWBURY TOWN, 58)"
1338,1954,NaN,NaN,BEDFORD TOWN,FIRST ROUND PROPER,8,"(LYE TOWN, 86)","(BEDFORD T, 86)","(RETFORD TOWN, 83)"
1339,1954,NaN,NaN,FERRYHILL ATHLETIC,FIRST ROUND PROPER,8,"(FERRYHILL ATH, 84)","(CRITTALL ATHLETIC, 74)","(ROTHWELL ATHLETIC, 74)"
1340,1954,NaN,NaN,GREAT YARMOUTH TOWN,FIRST ROUND PROPER,8,"(LYE TOWN, 86)","(DUDLEY TOWN, 86)","(SUTTON TOWN, 86)"


There appears to be a lot of team names truncated to T where T = Town, C where C = City, U where U = United and Ath = Athletic

Will check if there are any current matches we need to preserve and update the rest 

In [227]:
nl_merge4[nl_merge4.team.str.contains(' T$', regex=True)]

,year,pos,league,team,round,stage
1040,1981,9,SOUTHERN LEAGUE DIVISION 1 SOUTH,ADDLESTONE & WEYBRIDGE T,FIRST ROUND PROPER,8


#### Checking and updating 'T'  teams

In [228]:
mask_T = ~((nl_tab.year == 1981) & (nl_tab.team == 'ADDLESTONE & WEYBRIDGE T'))

In [229]:
mask_T.value_counts()

True     30572
False        1
dtype: int64

In [230]:
nl_tab.ix[mask_T, 'team'] = nl_tab[mask_T].team.str.replace(' T$',' TOWN')

#### Checking and updating 'U'  teams

In [231]:
nl_merge4[nl_merge4.team.str.contains(' U$', regex=True)]

,year,pos,league,team,round,stage


In [232]:
nl_tab['team'] = nl_tab['team'].str.replace(' U$',' UNITED')

#### Checking and updating 'C'  teams

In [233]:
nl_merge4[nl_merge4.team.str.contains(' C$', regex=True)]

,year,pos,league,team,round,stage


In [234]:
nl_tab['team'] = nl_tab['team'].str.replace(' C$',' CITY')

#### Checking and updating 'ATH'  teams

In [235]:
nl_merge4[nl_merge4.team.str.contains(' ATH$', regex=True)]

,year,pos,league,team,round,stage


In [236]:
nl_tab['team'] = nl_tab['team'].str.replace(' ATH$',' ATHLETIC')

In [237]:
nl_tab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30573 entries, 0 to 32531
Data columns (total 4 columns):
year      30573 non-null int64
pos       30573 non-null int64
league    30573 non-null object
team      30573 non-null object
dtypes: int64(2), object(2)
memory usage: 1.2+ MB


#### Merge test 5

In [238]:
nl_merge5 = pd.merge(nl_tab, nl_fac, how='right', on=['team', 'year'])

missing_teams5 = nl_merge5[nl_merge5.pos.isnull()].copy()

mt_plot5 = missing_teams5.year.value_counts().sort_index().reindex()

In [239]:
mt_plot_join['mt_plot5'] = mt_plot5

mt_plot_join.iplot(kind='bar', barmode='overlay', dimensions = (700,400))

In [240]:
%time missing_teams5['match1'], missing_teams5['match2'], missing_teams5['match3'] = zip(*missing_teams5.apply(near_matches, axis = 1))

Wall time: 7.61 s


Now exporting at finding and filling missing teams individually

In [241]:
missing_teams5.to_csv('missing_teams_1952-89_v1.csv', index = False)

All missing teams manually researched and updated (apart from RAWMARSH WELFARE, 1952 league position). This will be filled in when found. (provisionally 3rd)

In [243]:
nl_tab_aux = pd.read_csv('missing_teams_1952-89_v2.csv')[['year', 'pos', 'league', 'team']]

nl_tab_aux.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78 entries, 0 to 77
Data columns (total 4 columns):
year      78 non-null int64
pos       78 non-null int64
league    78 non-null object
team      78 non-null object
dtypes: int64(2), object(2)
memory usage: 3.0+ KB


In [245]:
nl_tab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30573 entries, 0 to 32531
Data columns (total 4 columns):
year      30573 non-null int64
pos       30573 non-null int64
league    30573 non-null object
team      30573 non-null object
dtypes: int64(2), object(2)
memory usage: 1.2+ MB


In [246]:
nl_tab = pd.concat([nl_tab, nl_tab_aux])

#### Merge test 6

In [248]:
nl_merge6 = pd.merge(nl_tab, nl_fac, how='right', on=['team', 'year'])

missing_teams6 = nl_merge6[nl_merge6.pos.isnull()].copy()

mt_plot6 = missing_teams6.year.value_counts().sort_index().reindex()

In [249]:
mt_plot_join['mt_plot6'] = mt_plot6

mt_plot_join.iplot(kind='bar', barmode='overlay', dimensions = (700,400))

In [250]:
missing_teams6

,year,pos,league,team,round,stage


Now looking at teams that have multiple matches and keeping the correct one 

Teams will be kept preferentially from 'higher' leagues, that is NORTHERN, SOUTHERN, ISTHMIAN, CONFERENCE etc whilst the  duplicate team in a 'regional' league will be the reserve team not elligible for the FA Cup

In [251]:
dupe_match = nl_merge6.groupby(['year', 'team']).filter(lambda x: len(x) > 1)

dupe_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411 entries, 2 to 1342
Data columns (total 6 columns):
year      411 non-null int64
pos       411 non-null int64
league    411 non-null object
team      411 non-null object
round     411 non-null object
stage     411 non-null float64
dtypes: float64(1), int64(2), object(3)
memory usage: 22.5+ KB


In [252]:
dupe_match.to_csv('dupe_match_1952-89_v1.csv', index = False)

In [253]:
de_duped = pd.read_csv('dupe_match_1952-89_v2.csv')

In [254]:
de_duped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202 entries, 0 to 201
Data columns (total 6 columns):
year      202 non-null int64
pos       202 non-null int64
league    202 non-null object
team      202 non-null object
round     202 non-null object
stage     202 non-null int64
dtypes: int64(3), object(3)
memory usage: 11.0+ KB


In [255]:
no_dupe = nl_merge6.groupby(['year', 'team']).filter(lambda x: len(x) == 1)

In [256]:
no_dupe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1014 entries, 0 to 1424
Data columns (total 6 columns):
year      1014 non-null int64
pos       1014 non-null int64
league    1014 non-null object
team      1014 non-null object
round     1014 non-null object
stage     1014 non-null float64
dtypes: float64(1), int64(2), object(3)
memory usage: 55.5+ KB


In [257]:
de_duped.groupby(['year', 'team']).filter(lambda x: len(x) > 1)  #checking no duplicates were missed

,year,pos,league,team,round,stage


In [258]:
fin_1952_89 = pd.concat([no_dupe, de_duped])

In [259]:
fin_1952_89.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1216 entries, 0 to 201
Data columns (total 6 columns):
year      1216 non-null int64
pos       1216 non-null int64
league    1216 non-null object
team      1216 non-null object
round     1216 non-null object
stage     1216 non-null float64
dtypes: float64(1), int64(2), object(3)
memory usage: 66.5+ KB


#### Final save of non-league 1952-89 merge

In [261]:
fin_1952_89.to_csv(r'C:\Users\David\Python\scrapy\facup\02_merge\fa_nl_merge_1952-89_v1.csv', index=False)